In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import demoMF 
#import dask.dataframe as dd
#from sklearn.utils.extmath import randomized_svd
#from sklearn.decomposition import TruncatedSVD

In [6]:
# = "ml-25m/genome-scores.csv"
#gtpath = "ml-25m/genome-tags.csv"
#path_tags = 'ml-/tags.csv'
path_ratings = 'ml-latest-small/ratings.csv'
path_movies = 'ml-latest-small/movies.csv'

In [7]:
#df_gscore = dd.read_csv(gspath)
#df_gtag = dd.read_csv(gtpath)
#df_tags = pd.read_csv(path_tags)
rating_df = pd.read_csv(path_ratings)
movie_df = pd.read_csv(path_movies)

In [8]:
combine_movie_rating = pd.merge(rating_df, movie_df, on='movieId')

In [10]:
columns = ['timestamp', 'genres']
combine_movie_rating = combine_movie_rating.drop(columns, axis=1)
combine_movie_rating.head(10)

,userId,movieId,rating,title
0,1,31,2.5,Dangerous Minds (1995)
1,7,31,3.0,Dangerous Minds (1995)
2,31,31,4.0,Dangerous Minds (1995)
3,32,31,4.0,Dangerous Minds (1995)
4,36,31,3.0,Dangerous Minds (1995)
5,39,31,3.0,Dangerous Minds (1995)
6,73,31,3.5,Dangerous Minds (1995)
7,88,31,3.0,Dangerous Minds (1995)
8,96,31,2.5,Dangerous Minds (1995)
9,110,31,4.0,Dangerous Minds (1995)


In [11]:
movie_ratingCount = (combine_movie_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )

In [12]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')

In [13]:
user_rating = rating_with_totalRatingCount.drop_duplicates(['userId','title'])

In [14]:
movie_user_rating_pivot = user_rating.pivot(index = 'userId', columns = 'title', values = 'rating').fillna(0)

In [15]:
X = movie_user_rating_pivot.values
dim = movie_user_rating_pivot.shape
dim

(671, 9064)

In [16]:
objMF = demoMF.ExplicitMF(X,60,0.1,0.1,verbose=True)

In [17]:
objMF.train(10)

	current iteration: 10


In [18]:
U = objMF.user_vecs
V = objMF.item_vecs
V.shape

(9064, 60)

In [19]:
matrix = V
corr = np.corrcoef(matrix)
corr.shape

(9064, 9064)

In [20]:
movie_title = movie_user_rating_pivot.columns
movie_title_list = list(movie_title)

In [21]:
coffey_hands = movie_title_list.index("Monsters, Inc. (2001)")

In [22]:
corr_coffey_hands  = corr[coffey_hands]
#corr_coffey_hands[:10]

In [24]:
list(movie_title[(corr_coffey_hands >= 0.7)])[::-1]

['Toy Story 2 (1999)',
 'Spider-Man (2002)',
 'Shrek 2 (2004)',
 'Shrek (2001)',
 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)',
 "Ocean's Eleven (2001)",
 'Monsters, Inc. (2001)',
 'Lord of the Rings: The Two Towers, The (2002)',
 'Lord of the Rings: The Return of the King, The (2003)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Incredibles, The (2004)',
 'Finding Nemo (2003)',
 'Chicken Run (2000)',
 'Catch Me If You Can (2002)']